In [2]:
import time
import random
time.sleep(random.random()*3)

In [3]:
import re

In [4]:
import pandas as pd

In [5]:
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

In [6]:
import requests
from bs4 import BeautifulSoup

In [7]:
import json

In [8]:
salary_ranges = ['0-70000','70000-120000','120000-999999']
# declare a list
job_urls = {}
salary_range_list = []

for salary_range in salary_ranges:
    job_urls[salary_range]=[]
    for page in range(1,40):
        # iteraterate through the 11 pages of jobs
        r = requests.get("https://www.seek.com.au/data-scientist-jobs?page={0}&salaryrange={1}&salarytype=annual".format(str(page),salary_range))

        # turn into a BeautifulSoup object
        soup = BeautifulSoup(r.text, 'lxml')

        # find quotes on page
        urls = soup.find_all(attrs={'data-automation':"jobTitle"})
        
        if urls != []: 
            job_urls[salary_range].extend(urls)
            list_of_ranges = [salary_range]*len(urls)
            salary_range_list.extend(list_of_ranges)

If there is one job which belongs to two or more salary ranges, the concatenated list will be a mess. Thus, I create different dataframes for different salary ranges and then combine them as one.

In [9]:
df_1 = pd.DataFrame()
df_1['Link'] = job_urls['0-70000']
df_1['Salary Range'] = '0-70000'

In [10]:
df_2 = pd.DataFrame()
df_2['Link'] = job_urls['70000-120000']
df_2['Salary Range'] = '70000-120000'

In [11]:
df_3 = pd.DataFrame()
df_3['Link'] = job_urls['120000-999999']
df_3['Salary Range'] = '120000-999999'

In [12]:
url_pd = df_1.append(df_2).append(df_3)

In [13]:
url_pd['Link_1'] =['https://www.seek.com.au' + re.search('(\/job\/[0-9]{8})',i.attrs['href']).group(0) for i in url_pd['Link']]

In [14]:
url_pd['Title'] = [i.text for i in url_pd['Link']]

In [15]:
url_pd = url_pd.drop('Link',axis='columns')

In [16]:
columns = ['Salary Range', 'Link', "Title"]
url_pd.columns=columns

In [17]:
url_pd.head()

,Salary Range,Link,Title
0,0-70000,https://www.seek.com.au/job/39746982,Data Scientist
1,0-70000,https://www.seek.com.au/job/39752938,Data Scientist Graduate
2,0-70000,https://www.seek.com.au/job/39744582,Data Scientist
3,0-70000,https://www.seek.com.au/job/39713902,Data Analyst / Associate Data Scientist
4,0-70000,https://www.seek.com.au/job/39719742,Graduate Business - Data Scientist


In [18]:
#test
r = requests.get('https://www.seek.com.au/job/39582387')
soup = BeautifulSoup(r.text,'lxml')
jd_data = soup.find_all(attrs={'data-automation':'server-state'})[0]
jd_raw= jd_data.text[3:].split('\n  ')[1][25:][:-1]

In [19]:
import re

def replace(string, substitutions):

    substrings = sorted(substitutions, key=len, reverse=True)
    regex = re.compile('|'.join(map(re.escape, substrings)))
    return regex.sub(lambda match: substitutions[match.group(0)], string)

In [20]:
job_listing_date=[]
job_expiry_date=[]
job_title = []
job_teaser=[]
job_advertiser=[]
job_area=[]
job_worktype=[]
job_classification=[]
job_salary=[]
job_salary_type=[]
job_description_details=[]
job_description_strong = []
job_location =[]

for index, job in enumerate(url_pd['Link']):
    # iterate through the 202 urls
    r = requests.get(job)
    
    # turn into a BeautifulSoup object
    soup = BeautifulSoup(r.text, 'lxml')
    
    # find job descriptions
    jd_data = soup.find_all(attrs={'data-automation':'server-state'})[0]
    jd_raw= jd_data.text[3:].split('\n  ')[1][25:][:-1]
    jd_dict = json.loads(jd_raw)
    jd_dashboard = jd_dict['jobdetails']['result']
    
    if jd_dashboard['listingDate']:
        job_listing_date.append(jd_dashboard['listingDate'])
    else:
        job_listing_date.append('N/A')
        
    if jd_dashboard['expiryDate']:
        job_expiry_date.append(jd_dashboard['expiryDate'])
    else:
        job_expiry_date.append('N/A')
        
    if jd_dashboard['title']:
        job_title.append(jd_dashboard['title'])
    else:
        job_title.append('N/A')
        
    if jd_dashboard['teaser']:
        job_teaser.append(jd_dashboard['teaser'])
    else:
        job_teaser.append('N/A')
        
    if jd_dashboard['advertiser']['description']:
        job_advertiser.append(jd_dashboard['advertiser']['description'])
    else:
        job_advertiser.append('N/A')
        
    if jd_dashboard['locationHierarchy']['area']:
        job_area.append(jd_dashboard['locationHierarchy']['area'])
    else:
        job_area.append('N/A')
        
    if jd_dashboard['workType']:
        job_worktype.append(jd_dashboard['workType'])
    else:
        job_worktype.append('N/A')
        
    if jd_dashboard['classification']['description']:
        job_classification.append(jd_dashboard['classification']['description'])
    else:
        job_classification.append('N/A')
        
    if jd_dashboard['salary']:
        job_salary.append(jd_dashboard['salary'])
    else:
        job_salary.append('N/A')
        
    if jd_dashboard['salaryType']:
        job_salary_type.append(jd_dashboard['salaryType'])
    else:
        job_salary_type.append('N/A')
        
    if jd_dashboard['locationHierarchy']['city']:
        job_location.append(jd_dashboard['locationHierarchy']['city'])
    else:
        job_location.append('N/A')
    
    jd_detail = soup.find(attrs={'data-automation':'mobileTemplate'})
    job_description_details.append(jd_detail)
    

In [21]:
url_pd['Listing Date'] = job_listing_date
url_pd['Expiry Date'] =job_expiry_date
url_pd['Job Title']=job_title
url_pd['Job Teaser']=job_teaser
url_pd['Advertiser'] = job_advertiser
url_pd['Area'] = job_area
url_pd['Work Type'] = job_worktype
url_pd['Classification'] = job_classification
url_pd['Salary'] = job_salary
url_pd['Salary Type'] = job_salary_type
url_pd['JD'] = job_description_details
url_pd['Location'] =job_location

In [22]:
job_description_clean=[]
job_description_strong = []
for i in range(0,len(url_pd['JD'])):
    strong_word = job_description_details[i].find_all('strong')
    strong_word_list = [u.get_text(strip=True) for u in strong_word if strong_word != False]
    job_description_strong.append(strong_word_list)

    string = job_description_details[i].get_text()
    repla = {u'\xa0':'  ',u'\xe2\x80\x9d':'  ', u'\n':'  '}
    job_description_clean.append(replace(string,repla))

In [23]:
url_pd['Job Description'] = job_description_clean
url_pd['Strong Words'] = job_description_strong 

In [31]:
url_pd.head()

,Salary Range,Link,Title,Listing Date,Expiry Date,Job Title,Job Teaser,Advertiser,Area,Work Type,Classification,Salary,Salary Type,JD,Location,Job Description,Strong Words
0,0-70000,https://www.seek.com.au/job/39746982,Data Scientist,2019-08-20T05:56:16.000Z,2019-09-19T13:59:59.000Z,Data Scientist,Data Scientist Role with Deloitte Consulting -...,Deloitte,CBD & Inner Suburbs,Full Time,Information & Communication Technology,N/A,AnnualPackage,[[· Salary packaging - to suit your personal a...,Brisbane,· Salary packaging - to suit your personal and...,"[About the team, , What will your typical day ..."
1,0-70000,https://www.seek.com.au/job/39752938,Data Scientist Graduate,2019-08-21T01:28:04.000Z,2019-09-20T01:28:03.000Z,Data Scientist Graduate,To strengthen the future of IT we are looking ...,Wipro Limited,"CBD, Inner West & Eastern Suburbs",Full Time,Science & Technology,N/A,AnnualPackage,"[[[About Wipro]], , [Wipro is a leading globa...",Sydney,"About Wipro Wipro is a leading global IT, cons...","[About Wipro, About the role, starting mid-Sep..."
2,0-70000,https://www.seek.com.au/job/39744582,Data Scientist,2019-08-20T03:32:33.000Z,2019-09-19T13:59:59.000Z,Data Scientist,Data Scientist role with Deloitte Consulitng -...,Deloitte,"CBD, Inner & Western Suburbs",Full Time,Information & Communication Technology,N/A,AnnualPackage,[[· Salary packaging - to suit your personal a...,Perth,· Salary packaging - to suit your personal and...,"[What will your typical day look like?, About ..."
3,0-70000,https://www.seek.com.au/job/39713902,Data Analyst / Associate Data Scientist,2019-08-15T06:18:09.000Z,2019-09-14T06:18:08.000Z,Data Analyst / Associate Data Scientist,Join a fast paced artificial intelligence and ...,Elula,"CBD, Inner West & Eastern Suburbs",Full Time,Information & Communication Technology,N/A,AnnualPackage,"[[[About Elula and our team]], , [Elula is an...",Sydney,About Elula and our team Elula is an exciting ...,"[About Elula and our team, About the opportuni..."
4,0-70000,https://www.seek.com.au/job/39719742,Graduate Business - Data Scientist,2019-08-16T02:24:49.000Z,2019-09-15T13:59:59.000Z,Graduate Business - Data Scientist,Exceptional opportunity for an energetic and m...,Adelaide Brighton Ltd,N/A,Full Time,Information & Communication Technology,N/A,AnnualPackage,"[[], [[The role]], [Our Graduate Program has b...",Gold Coast,The roleOur Graduate Program has been designed...,"[The role, Responsibilities, What you will nee..."


In [25]:
data_scientist_jobs = url_pd.copy()

In [26]:
data_scientist_jobs.to_csv('data_scientist_scraping.csv')